In [66]:
%load_ext autoreload 
%autoreload 2
import sys
sys.tracebacklimit = 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
from pettingzoo.test import parallel_api_test
from solution.custom_gym import CustomGymEnviornment
from solution.trainer import *

In [68]:
from core.agent import *
from core.world import * 
from core.map import *
from core.render import * 
from core.env_params import * 
from core.skill import * 
from core.models import *
from core.env_params import *

In [69]:
def initialize_swarm(world : World):
    swarm = [Agent() for i in range(SWARM_SIZE)]
    skill_initializer = DefaultSkillInitializer(num_skills = PRODUCT_TYPES)
    utility_initializer = UtilitySampler()

    for agent in swarm:
        skill_initializer.forward(agent)
        utility_initializer.forward(agent)
        world.add_agent(agent)
    swarm = initialize_positions_randomly(world, swarm)
    

In [70]:
world = World(dims = WORLD_DIMS,
              swarm_initialzier= initialize_swarm,
              resource_generator= RandomMapGenerator(RESOURCE_TYPES),
              energy_model=EnergyModel(),
              chemistry_model= ChemistryModel()
              )
world.reset()

In [71]:
custom_gym : CustomGymEnviornment = CustomGymEnviornment(world)

In [72]:
custom_gym.reset(42)

({1: {'vision': array([[0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0]])},
  2: {'vision': array([[0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 5, 5, 5, 0, 0],
          [0, 0, 5, 5, 5, 0, 0],
          [0, 0, 5, 5, 5, 0, 0]])},
  3: {'vision': array([[0, 0, 1, 1, 1, 1, 1],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0]])},
  4: {'vision': array([[0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0]])},
  5: {'visio

### Testing

In [73]:
parallel_api_test(custom_gym, num_cycles=1_000_000)

Passed Parallel API test


In [74]:
train_loop(custom_gym, games=10, seed=0)

Starting training on thesis.
Traceback (most recent call last):
  File "c:\Users\Joaquin\miniconda3\envs\thesis\Lib\site-packages\supersuit\vector\multiproc_vec.py", line 76, in async_loop
    observations, infos = vec_env.reset(seed=data[0], options=data[1])
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Joaquin\miniconda3\envs\thesis\Lib\site-packages\supersuit\vector\concat_vec_env.py", line 45, in reset
    _obs, _info = self.vec_envs[i].reset(options=options)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Joaquin\miniconda3\envs\thesis\Lib\site-packages\supersuit\vector\markov_vector_wrapper.py", line 57, in reset
    _observations, infos = self.par_env.reset(seed=seed, options=options)
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Joaquin\miniconda3\envs\thesis\Lib\site-packages\pettingzoo\utils\conversions.py", line 187, in reset
    infos = dict(**self.aec_env.infos

TypeError: keywords must be strings